In [1]:
import pandas as pd

class CustomerDataLoader:
    def __init__(self):
        self.raw_data = None

    def load_from_sources(self, csv_paths: dict):
        self.raw_data = {
            'customers': pd.read_csv(csv_paths['customers']),
            'transactions': pd.read_csv(csv_paths['transactions']),
            'sentiment': pd.read_csv(csv_paths['sentiment'])
        }
        return self

    def create_unified_profile(self):
        merged = pd.merge(
            self.raw_data['customers'],
            self.raw_data['transactions'],
            on='customer_id'
        )
        return merged

In [2]:
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer

class FineTuner:
    def __init__(self, base_model="meta-llama/Llama-2-7b-chat-hf"):
        self.model = AutoModelForCausalLM.from_pretrained(
            base_model,
            load_in_4bit=True,
            device_map="auto"
        )
        self.tokenizer = AutoTokenizer.from_pretrained(base_model)

    def apply_lora(self):
        """Parameter-efficient adaptation"""
        config = LoraConfig(
            r=8,
            lora_alpha=32,
            target_modules=["q_proj", "v_proj"],
            lora_dropout=0.05,
            bias="none",
            task_type="CAUSAL_LM"
        )
        return get_peft_model(self.model, config)

    def train(self, dataset, epochs=1):
        """Lightweight training loop"""
        training_args = TrainingArguments(
            output_dir="./results",
            per_device_train_batch_size=4,
            gradient_accumulation_steps=4,
            learning_rate=2e-5,
            num_train_epochs=epochs,
            fp16=True
        )

        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=dataset
        )
        trainer.train()

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

class FinancialRAG:
    def __init__(self):
        self.embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

    def build_index(self, documents):
        return FAISS.from_documents(documents, self.embeddings)

    def retrieve(self, query, index, k=3):
        return index.similarity_search(query, k=k)